The Piotroski F-Score is often used by value investors to avoid the so-called value trap, that is, the trap of buying cheap stocks that are only cheap because they are in trouble.
The Piotroski F-Score is a composite factor comprising multiple sub-factors. It is calculated as the sum of 9 true/false criteria, resulting in a possible score of 0-9. Points are awarded as follows:

- Profitability:
    * 1 point if return on assets (ROA) is positive in the current year
    * 1 point if operating cash flow is positive in the current year
    * 1 point if ROA is higher in the current period than in the previous year
    * 1 point if cash flow from operations to total assets is higher than ROA (quality of earnings, accruals)
- Leverage, Liquidity and Source of Funds:
    * 1 point if long term debt is lower in the current period than in the previous year (decreased leverage, change in long-term leverage ratio)
    * 1 point if current ratio is higher in the current period than in the previous year (more liquidity)
    * 1 point if no new shares were issued in the last year (lack of dilution, change in the number of shares)
- Operating Efficiency:
    * 1 point if gross margin is higher in the current period than in the previous year
    * 1 point if asset turnover ratio is higher in the current period than in the previous year

In [1]:
import sys
sys.path.append("/Users/weizhang/Documents/_GIT/nyu-hedge-fund-strategies/nyu-hedge-fund-strategies")
import nasdaqdatalink as quandl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from common import factor_distribution
from sharadar import equity_prices, fundamental_signal
from scipy.stats import pearsonr

In [2]:
quandl.ApiConfig.api_key = 'NRvcyMwNMXZ2ooDSM3nw'
universe = quandl.get_table('SHARADAR/SP500', action='historical', date='2022-12-31')

In [3]:
fundamentals = quandl.get_table('SHARADAR/SF1', datekey={'gte':'2022-07-31','lte':'2022-12-31'}, dimension="ART", ticker=",".join(universe['ticker'].to_list()))
fundamentals = fundamentals.drop_duplicates("ticker", keep="first")
fundamentals = fundamental_signal.operating_margin(fundamentals)

In [4]:
fundamentals['roa'] = fundamentals['netinc'] / fundamentals['assets']
fundamentals['ocf'] = fundamentals['ebit'] - fundamentals['depamor'] - fundamentals['taxliabilities']
fundamentals['accruals'] = fundamentals['ocf'] - fundamentals['netinc']

In [ ]:
p_fundamentals = quandl.get_table('SHARADAR/SF1', datekey={'gte':'2021-07-31','lte':'2021-12-31'}, dimension="ART", ticker=",".join(universe['ticker'].to_list()))
p_fundamentals = p_fundamentals.drop_duplicates("ticker", keep="first")
p_fundamentals = fundamental_signal.operating_margin(p_fundamentals)


In [ ]:
p_fundamentals['roa'] = p_fundamentals['netinc'] / p_fundamentals['assets']

In [ ]:

fundamentals = pd.merge(left=fundamentals, right=p_fundamentals, left_on="ticker", right_on="ticker", how="inner", suffixes=('_c', '_p'))
fundamentals["roa_change"] = fundamentals["roa_c"] - fundamentals["roa_p"]
fundamentals["dilution"] = fundamentals["sharesbas_c"] - fundamentals["sharesbas_p"]
fundamentals["grossmargin_change"] = fundamentals["grossmargin_c"] - fundamentals["grossmargin_p"]